In [1]:
import os
from dotenv import load_dotenv
from pymongo import MongoClient

load_dotenv("../env")
client = MongoClient(os.environ.get("MONGO_DB_CONNECTION"))
collection = client.get_database("prismai").get_collection("collected_items")

In [2]:
from itertools import batched

from datasets import Dataset
from tqdm.auto import tqdm

In [3]:
from transition_scores.scorer import OnnxTransitionScorer

scorer = OnnxTransitionScorer(
    "/hot_storage/models/onnx/gpt2_onnx_o4/",
    batch_size=4,
    device="cuda",
)

2025-01-21 20:08:38.382516967 [W:onnxruntime:, transformer_memcpy.cc:74 ApplyImpl] 24 Memcpy nodes are added to the graph main_graph for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.
2025-01-21 20:08:38.386832199 [W:onnxruntime:, session_state.cc:1168 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2025-01-21 20:08:38.386839219 [W:onnxruntime:, session_state.cc:1170 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.


In [4]:
total = collection.count_documents({})

for batch in batched(
    tqdm(
        collection.find(
            projection=[
                "text",
                # "chunks",
            ],
            batch_size=128,
        ),
        total=total,
    ),
    512,
):
    dataset = Dataset.from_list([dd | {"_id": str(dd["_id"])} for dd in batch])
    dataset = dataset.filter(lambda x: len(x["text"]) > 0)
    for scores in scorer.process(dataset=dataset, top_k=100):
        pass

  0%|          | 0/578610 [00:00<?, ?it/s]

Filter:   0%|          | 0/512 [00:00<?, ? examples/s]

Parameter 'function'=<bound method TransitionScorerABC._tokenize of <transition_scores.scorer.OnnxTransitionScorer object at 0x7aeaf1914bf0>> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/497 [00:00<?, ? examples/s]

2025-01-21 20:09:05.739189880 [E:onnxruntime:, sequential_executor.cc:516 ExecuteKernel] Non-zero status code returned while running MatMul node. Name:'/lm_head/MatMul' Status Message: /onnxruntime_src/onnxruntime/core/framework/bfc_arena.cc:376 void* onnxruntime::BFCArena::AllocateRawInternal(size_t, bool, onnxruntime::Stream*, bool, onnxruntime::WaitNotificationFn) Failed to allocate memory for requested buffer of size 340943616



RuntimeError: Error in execution: Non-zero status code returned while running MatMul node. Name:'/lm_head/MatMul' Status Message: /onnxruntime_src/onnxruntime/core/framework/bfc_arena.cc:376 void* onnxruntime::BFCArena::AllocateRawInternal(size_t, bool, onnxruntime::Stream*, bool, onnxruntime::WaitNotificationFn) Failed to allocate memory for requested buffer of size 340943616


In [ ]:
from bson.dbref import DBRef

from transition_scores.data import LogProbs
from transition_scores.mongo import TextTransitionScore, TransitionScoreItem

dict(
    TransitionScoreItem(
        DBRef("a", "b"),
        "gpt2",
        "onnx",
        TextTransitionScore([LogProbs(0, 1.0, [0], [1.0])]),
    )
)